In [1]:
pip install --upgrade pip

  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/15/aa/3f4c7bcee2057a76562a5b33ecbd199be08cdb4443a02e26bd2c3cf6fc39/pip-23.3.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 4.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pip
    Found existing installation: pip 23.2.1
    Uninstalling pip-23.2.1:
      Successfully uninstalled pip-23.2.1
Note: you may need to restart the kernel to use updated packages.


In [2]:
!pip install pinecone-client pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.0/201.0 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.5/502.5 kB 11.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.3/78.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 346.6/346.6 kB 10.5 MB/s eta 0:00:00


In [3]:
from pinecone import Pinecone
import os

In [4]:
# initialize connection to pinecone (get API key at app.pc.io)
api_key = os.environ.get('PINECONE_API_KEY') or 'PINECONE_API_KEY'
environment = os.environ.get('PINECONE_ENVIRONMENT') or 'PINECONE_ENVIRONMENT'

# configure client
pc = Pinecone(api_key=api_key)

In [5]:
pc.list_indexes()

{'indexes': [{'dimension': 3,
              'host': 'stock-prices-9atubxs.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'stock-prices',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [6]:
# List indexes
indexes = pc.list_indexes()
print(indexes[0].name)

stock-prices


In [7]:
# Describe index
pc.describe_index(indexes[0].name)

{'dimension': 3,
 'host': 'stock-prices-9atubxs.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'stock-prices',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [8]:
#create index
new_index_name = "stock-prices"
dimension = 3
metric = "cosine"

In [9]:
# if exists delete
for index in pc.list_indexes().indexes:
    if index.name == new_index_name:
        pc.delete_index(new_index_name)

In [10]:
from pinecone import PodSpec
pc.create_index(
    name=new_index_name, 
    dimension=dimension, 
    metric=metric, 
    spec=PodSpec(
        environment="gcp-starter",
        pod_type="starter",
        pods=1,
        replicas=1,
        shards=1
))

In [11]:
pc.describe_index(new_index_name)

{'dimension': 3,
 'host': 'stock-prices-9atubxs.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'stock-prices',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [12]:
index = pc.Index(name=new_index_name)

In [13]:
index

In [14]:
# Insert some data
# [(id1, vecvtor1), (id2, vector2)]
index.upsert(
    [
        ("TSLA", [1., 1., 1.]),
        ("IBM",  [1., 2., 3.]),
        ("GM",   [2., 1., 1.]),
        ("AMZN", [3., 2., 1.]),
        ("C",    [2., 2., 2.]),
    ]
)

{'upserted_count': 5}

In [15]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [16]:
# Add meta data
index.upsert(
    [
        ("TSLA", [1., 1., 1.], {"stock_name":"Tesla", "sector":"automotive"}),
    ]
)

{'upserted_count': 1}

In [17]:
# insert data from a dataframe
data = {
    "id": ["NIO","v"],
    "vector": [
        [4., 4., 4.],
        [2., 2., 5.]
    ]
}
data

{'id': ['NIO', 'v'], 'vector': [[4.0, 4.0, 4.0], [2.0, 2.0, 5.0]]}

In [20]:
import pandas as pd
df = pd.DataFrame(data)
df

,id,vector
0,NIO,"[4.0, 4.0, 4.0]"
1,v,"[2.0, 2.0, 5.0]"
